# Configurações Iniciais para treinamento de modelo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import mlflow
import mlflow.sklearn
import os
import joblib
from pathlib import Path
import sys
import logging
from datetime import datetime
from fiap import LoggerManager

# Defina o caminho absoluto onde quer salvar os mlruns
print(f'Mlflow path: {mlflow.get_tracking_uri()}')
mlflow_path = Path.cwd().parent / 'mlruns'
# Cria a pasta se não existir
os.makedirs(mlflow_path, exist_ok=True)

# Diz pro MLflow usar esse caminho
mlflow.set_tracking_uri(f'file:///{mlflow_path}')

print(f'MLflow tracking URI configurada para: {mlflow.get_tracking_uri()}')

RANDOM_STATE = 345

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
	sys.path.insert(0, str(src_path))

# Logger customizado
log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='model_train')

logging.info('=' * 50)
logging.info(f'Início do treinamento: {datetime.now()}')

# Carregar dados

In [ ]:
logging.info('Carregar arquivo de dados processados')
df = pd.read_csv(Path.cwd().parent / 'data' / 'processed_data.csv')
logging.info(f'Dados carregados: {df.shape[0]} linhas, {df.shape[1]} colunas')

logging.info('Verificar tipos e valores nulos')
logging.info(f'{df.dtypes}')
logging.info(f'Valores nulos por coluna:\n{df.isnull().sum()}')

# Separar e normalizar dados

In [ ]:
X = df.drop(columns=['defasagem'])
y = df['defasagem']

# =====================================================
# Normalização
# =====================================================
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
logging.info('Dados normalizados com MinMaxScaler')
# salvar o scaler para uso futuro
joblib.dump(scaler, Path.cwd().parent / 'ml_models' / 'scaler.pkl')
logging.info(f"Scaler salvo em: {Path.cwd().parent / 'ml_models' / 'scaler.pkl'}")

In [ ]:
from fiap.utils.model_train import log_extreme_examples

log_extreme_examples(y, X)

# Separar em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
	X_scaled, y, test_size=0.2, random_state=RANDOM_STATE
)
logging.info(f'Tamanho do treino: {X_train.shape[0]} exemplos')
logging.info(f'Tamanho do teste: {X_test.shape[0]} exemplos')

# Treinamento do modelo

In [ ]:
from fiap.utils.model_train import treinar_modelos

# =====================================================
# Modelos e grids de hiperparâmetros
# =====================================================
modelos = {
	'Regressão Linear': LinearRegression(),
	'Árvore de Decisão': DecisionTreeRegressor(random_state=RANDOM_STATE),
	'Random Forest': RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
	'HistGradientBoosting': HistGradientBoostingRegressor(random_state=RANDOM_STATE),
}

param_grids = {
	'Regressão Linear': {'fit_intercept': [True, False], 'positive': [True, False]},
	'Árvore de Decisão': {
		'max_depth': [None, 5, 10, 15],
		'min_samples_split': [2, 5, 10],
		'min_samples_leaf': [1, 2, 4],
	},
	'Random Forest': {
		'n_estimators': [100, 200, 300],
		'max_depth': [None, 5, 10],
		'min_samples_split': [2, 5],
		'min_samples_leaf': [1, 2],
	},
	'HistGradientBoosting': {
		'max_iter': [100, 200],
		'max_depth': [None, 5, 10],
		'learning_rate': [0.01, 0.1, 0.2],
		'min_samples_leaf': [20, 50],
	},
}

df_resultados, melhor_modelo_geral = treinar_modelos(
	X_train=X_train,
	X_test=X_test,
	y_train=y_train,
	y_test=y_test,
	modelos=modelos,
	param_grids=param_grids,
	experiment_name='Defasagem Escolar - Modelos',
	model_dir=Path.cwd().parent / 'ml_models',
	random_state=RANDOM_STATE,
)

# Validação de extremos novamente para ver se o modelo está coerente

In [ ]:
logging.info('Validando predições para alunos com defasagem extrema (-2 e 2)')
for val, idx_list in zip([-2, 2], [y[y == -2].index, y[y == 2].index]):
	if len(idx_list) > 0:
		i = idx_list[0]
		logging.info(f'Aluno com defasagem {val}: X={X.iloc[i].to_dict()}, y={y.iloc[i]}')
		logging.info(f'Predição do modelo: {melhor_modelo_geral.predict([X_scaled[i]])[0]:.4f}')